In [1]:
#import local functions

In [14]:
import sys
repo_dir = '/Users/michaelkranz/Documents/restaurant-app/'
sys.path.append(repo_dir)

#for now, using locally defined tokenize text --> will modularize later
#from src.util import tokenize_text

In [4]:
from jupyter_dash import JupyterDash

import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

import networkx as nx

In [13]:
def tokenize_text(text_str,nlp_obj):
    '''
    TODO: make in module
    use spacy to separate text into words
    (ie tokenization)
    and return the lemmatization 
    (ie feet for footing and foot)
    for only nouns and adjectives
    
    TODO: refine methodology
    '''
    spacy_doc = nlp_obj(text_str)
    
    tokenized_doc = [
        token.lemma_
        for token in spacy_doc
        ]
    
    return tokenized_doc
    #return spacy_doc

In [5]:
#needed for Jupyter lab to detect proxy config
#JupyterDash.infer_jupyter_proxy_config()

In [6]:
graph = nx.read_gpickle(f"{repo_dir}/data/champaign_restaurant_review.gpickle")

In [7]:
node_df = (
    pd.DataFrame(
        dict(graph.nodes(data=True))
        )
        .transpose()
)

In [9]:
tokenize_text('i want Kentucky-Fried Chicken spicy chiken recipe so i can be satisfied')

[('Kentucky', 'PROPN'),
 ('Fried', 'PROPN'),
 ('Chicken', 'PROPN'),
 ('spicy', 'NOUN'),
 ('chiken', 'NOUN'),
 ('recipe', 'NOUN'),
 ('satisfied', 'ADJ')]

In [10]:
tokenize_text('in America, we are lucky')

[('America', 'PROPN'), ('lucky', 'ADJ')]

In [11]:
node_df.categories.iloc[100]

'American (New), Restaurants, Local Flavor'

In [12]:
tokenize_text(node_df.categories.iloc[2])

[('Mexican', 'PROPN'), ('restaurant', 'NOUN')]

In [7]:
#tokenize and lemmatize query
#tokenize and lemmatize categories
#make free text search bar and table
#click callback and update another table based on selections

,business_name,stars,address,review_count,categories
0,Fiesta Ranchera,2,1805 S Neil St,4,"Mexican, Restaurants"
1,Po' Boys Restaurant,4,"202 E University Ave, Ste C",71,"Pizza, Barbeque, Restaurants"
2,La Fiesta Mexican Restaurant,3.5,1717 Philo Rd,12,"Mexican, Restaurants"
3,BIGGBY COFFEE,4,401 S Mattis Ave,17,"Coffee & Tea, Bakeries, Cafes, Restaurants, Ba..."
4,Rainbow Garden,3,202 E University,41,"Restaurants, Chinese"


In [38]:
categories.loc['Mexican']

28

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server